In [7]:
import pandas as pd
import numpy as np
import os 
#import torch
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors  # K-Nearest Neighbors
from sklearn.linear_model import LogisticRegression  # Logistic Regression
from sklearn.tree import DecisionTreeClassifier  # Decision Tree
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# from scraping import get_ingredients
from SMILES import CIRconvert
import pubchempy as pcp
import cirpy
import ast
import time
import csv
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
# import faiss

In [8]:
df = pd.read_csv(r'C:\Users\tiffa\Documents\HackUMass25\skincare_products_clean.csv')
df

,product_name,product_url,product_type,clean_ingreds,price
0,The Ordinary Natural Moisturising Factors + HA...,https://www.lookfantastic.com/the-ordinary-nat...,Moisturiser,"['capric triglyceride', 'cetyl alcohol', 'prop...",£5.20
1,CeraVe Facial Moisturising Lotion SPF 25 52ml,https://www.lookfantastic.com/cerave-facial-mo...,Moisturiser,"['homosalate', 'glycerin', 'octocrylene', 'eth...",£13.00
2,The Ordinary Hyaluronic Acid 2% + B5 Hydration...,https://www.lookfantastic.com/the-ordinary-hya...,Moisturiser,"['sodium hyaluronate', 'sodium hyaluronate', '...",£6.20
3,AMELIORATE Transforming Body Lotion 200ml,https://www.lookfantastic.com/ameliorate-trans...,Moisturiser,"['ammonium lactate', 'c12-15', 'glycerin', 'pr...",£22.50
4,CeraVe Moisturising Cream 454g,https://www.lookfantastic.com/cerave-moisturis...,Moisturiser,"['glycerin', 'cetearyl alcohol', 'capric trigl...",£16.00
...,...,...,...,...,...
1133,Elemis Life Elixirs Embrace Bath and Shower El...,https://www.lookfantastic.com/elemis-life-elix...,Bath Oil,"['prunus amygdalus dulcis', 'tipa-laureth sulf...",£55.00
1134,Love Boo Splendidly Soothing Bath Soak (250ml),https://www.lookfantastic.com/love-boo-splendi...,Bath Oil,"['sodium lauroyl', 'sodium cocoamphoacetate', ...",£10.99
1135,Elemis Life Elixirs Fortitude Bath and Shower ...,https://www.lookfantastic.com/elemis-life-elix...,Bath Oil,"['prunus amygdalus dulcis', 'tipa-laureth sulf...",£55.00
1136,Connock London Kukui Oil Soothing Bath & Showe...,https://www.lookfantastic.com/connock-london-k...,Bath Oil,"['capric triglyceride', 'peg-40 sorbitan perol...",£36.00


In [9]:
smiles_df = pd.read_csv(r'smiles_with_no_nulls.csv')
smiles_df

,Unnamed: 0,product_name,product_type,clean_ingreds,price,SMILES
0,0,The Ordinary Natural Moisturising Factors + HA...,Moisturiser,"['capric triglyceride', 'cetyl alcohol', 'prop...",£5.20,"['CCCCCCCCCCCCCCCCO', 'CCC(O)O', 'CCCCCCCCCCCC..."
1,1,CeraVe Facial Moisturising Lotion SPF 25 52ml,Moisturiser,"['homosalate', 'glycerin', 'octocrylene', 'eth...",£13.00,"['CC1CC(CC(C)(C)C1)OC(=O)c2ccccc2O', 'OCC(O)CO..."
2,2,The Ordinary Hyaluronic Acid 2% + B5 Hydration...,Moisturiser,"['sodium hyaluronate', 'sodium hyaluronate', '...",£6.20,['[Na+].CC(=O)N[C@H]1[C@H](O)O[C@H](CO)[C@@H](...
3,3,AMELIORATE Transforming Body Lotion 200ml,Moisturiser,"['ammonium lactate', 'c12-15', 'glycerin', 'pr...",£22.50,"['[NH4+].CC(O)C([O-])=O', 'OCC(O)CO', 'CCCCCCC..."
4,4,CeraVe Moisturising Cream 454g,Moisturiser,"['glycerin', 'cetearyl alcohol', 'capric trigl...",£16.00,"['OCC(O)CO', 'CCCCCCCCCCCCCCCCO.CCCCCCCCCCCCCC..."
...,...,...,...,...,...,...
1117,1133,Elemis Life Elixirs Embrace Bath and Shower El...,Bath Oil,"['prunus amygdalus dulcis', 'tipa-laureth sulf...",£55.00,"['CC(O)CO', 'CCCCCCCC\\C=C/CCCCCCCC(=O)OCC(O)[..."
1118,1134,Love Boo Splendidly Soothing Bath Soak (250ml),Bath Oil,"['sodium lauroyl', 'sodium cocoamphoacetate', ...",£10.99,['CCCCCCCCCCCCCCCCCCOC(=O)CC(O)(CC([O-])=O)C([...
1119,1135,Elemis Life Elixirs Fortitude Bath and Shower ...,Bath Oil,"['prunus amygdalus dulcis', 'tipa-laureth sulf...",£55.00,"['CC(O)CO', 'CCCCCCCC\\C=C/CCCCCCCC(=O)OCC(O)[..."
1120,1136,Connock London Kukui Oil Soothing Bath & Showe...,Bath Oil,"['capric triglyceride', 'peg-40 sorbitan perol...",£36.00,"['CCCCCCCCCCCCCCCCCCOCC(C)O', 'O=C(OCc1ccccc1)..."


In [10]:
# Generate fingerprints of all the molecules and store them in a df
# all_ingredients_list is the column containing the list of smiles representations of the ingredients of a skincare product
skipped = []
def generate_dataset_fingerprints(df):
    all_fingerprint_list = [] # 2d array that contains the lists of fingerprints of every row
    all_SMILES_list = df['SMILES']
    i = 0
    for smiles_list in all_SMILES_list:
        # make sure that it's a list and not a string
        smiles_list = ast.literal_eval(smiles_list)
        smiles_list = [n.strip() for n in smiles_list]

        # Convert SMILES to RDKit Molecule objects 
        single_fp_list = []
        for r in smiles_list:
            mol = Chem.MolFromSmiles(r)
            try:
                single_fp_list.append(AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048))
            except:
                skipped.append(i)
                continue
        print(single_fp_list)
        i += 1
        all_fingerprint_list.append(single_fp_list)
    return all_fingerprint_list

In [11]:
all_fingerprints = generate_dataset_fingerprints(smiles_df) # 2d array that contains the lists of fingerprints of every row
all_fingerprints

[<rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x000001A68B36DDF0>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x000001A68C7CC8F0>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x000001A68C74A850>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x000001A68C74A580>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x000001A68C74A0D0>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x000001A68C74A7B0>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x000001A68C7CF2B0>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x000001A68C7CF440>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x000001A68C7CF080>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x000001A68C7CF800>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x000001A68C7CF3A0>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x000001A68C7CF580>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect obj

[[<rdkit.DataStructs.cDataStructs.ExplicitBitVect at 0x1a68b36ddf0>,
 ...]

In [12]:
fingerprints_df = pd.DataFrame({
    'Fingerprints': all_fingerprints})
fingerprints_df

,Fingerprints
0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,"[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,"[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
...,...
1117,"[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1118,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,..."
1119,"[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1120,"[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,..."


In [13]:
# put together smiles and fingerprints dataframes
merged_df = pd.concat([smiles_df, fingerprints_df], axis=1)
merged_df

,Unnamed: 0,product_name,product_type,clean_ingreds,price,SMILES,Fingerprints
0,0,The Ordinary Natural Moisturising Factors + HA...,Moisturiser,"['capric triglyceride', 'cetyl alcohol', 'prop...",£5.20,"['CCCCCCCCCCCCCCCCO', 'CCC(O)O', 'CCCCCCCCCCCC...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,1,CeraVe Facial Moisturising Lotion SPF 25 52ml,Moisturiser,"['homosalate', 'glycerin', 'octocrylene', 'eth...",£13.00,"['CC1CC(CC(C)(C)C1)OC(=O)c2ccccc2O', 'OCC(O)CO...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
2,2,The Ordinary Hyaluronic Acid 2% + B5 Hydration...,Moisturiser,"['sodium hyaluronate', 'sodium hyaluronate', '...",£6.20,['[Na+].CC(=O)N[C@H]1[C@H](O)O[C@H](CO)[C@@H](...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,3,AMELIORATE Transforming Body Lotion 200ml,Moisturiser,"['ammonium lactate', 'c12-15', 'glycerin', 'pr...",£22.50,"['[NH4+].CC(O)C([O-])=O', 'OCC(O)CO', 'CCCCCCC...","[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,4,CeraVe Moisturising Cream 454g,Moisturiser,"['glycerin', 'cetearyl alcohol', 'capric trigl...",£16.00,"['OCC(O)CO', 'CCCCCCCCCCCCCCCCO.CCCCCCCCCCCCCC...","[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
...,...,...,...,...,...,...,...
1117,1133,Elemis Life Elixirs Embrace Bath and Shower El...,Bath Oil,"['prunus amygdalus dulcis', 'tipa-laureth sulf...",£55.00,"['CC(O)CO', 'CCCCCCCC\\C=C/CCCCCCCC(=O)OCC(O)[...","[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1118,1134,Love Boo Splendidly Soothing Bath Soak (250ml),Bath Oil,"['sodium lauroyl', 'sodium cocoamphoacetate', ...",£10.99,['CCCCCCCCCCCCCCCCCCOC(=O)CC(O)(CC([O-])=O)C([...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,..."
1119,1135,Elemis Life Elixirs Fortitude Bath and Shower ...,Bath Oil,"['prunus amygdalus dulcis', 'tipa-laureth sulf...",£55.00,"['CC(O)CO', 'CCCCCCCC\\C=C/CCCCCCCC(=O)OCC(O)[...","[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1120,1136,Connock London Kukui Oil Soothing Bath & Showe...,Bath Oil,"['capric triglyceride', 'peg-40 sorbitan perol...",£36.00,"['CCCCCCCCCCCCCCCCCCOCC(C)O', 'O=C(OCc1ccccc1)...","[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,..."


In [14]:
merged_df.to_csv('fingerprints_and_smiles.csv', index=False)

In [15]:
merged_df.shape

(1122, 7)

In [26]:
def calculate_top_5_similar_products(user_fingerprint_list, df):
    # Calculate Tanimoto similarity of each of the fingerprints of the user's product against every product's fingerprints
    tanimoto = {}
    i=0
    curr_row_index = 0
    curr_product_list_len = 0
    for product_fp_row in df['Fingerprints']: # product_fp_row is of type 2d list
        total_similarity = [0]*len(user_fingerprint_list) # Calculate the total sum of Tanimoto similarities between user's and product's fingerprints.
        for user_fp in user_fingerprint_list: # user_fp is of type list (fingerprint of one of the user's ingredients)
            for product_fp in product_fp_row: # product_fp is of tyoe list (fingerprint of one of our products' ingredients)
                curr_product_list_len = len(product_fp_row)
                similarity = DataStructs.TanimotoSimilarity(user_fp, product_fp)
                total_similarity[i] += similarity # total_similarity[i] is the sum of the similarity of each ingredient from the each of the user's ingredients against the product
                num_product_ingredients = len(product_fp_row)
            i += 1
        for j in range(len(total_similarity)):
            total_similarity[j] = total_similarity[j] / curr_product_list_len
        avg_tanimoto = sum(total_similarity) / len(total_similarity) # normalization
        key = df.iloc[curr_row_index, 1] # gets name of the current product being compared to and stores it as a key
        tanimoto[key] = avg_tanimoto
        # prevent out of bounds error
        i = 0
        curr_row_index += 1
    return dict(sorted(tanimoto.items(), key=lambda item: item[1], reverse=True)[:5])

In [ ]:
# # Convert the SMILES to fingerprints and calculates Tanimoto similarity
# # Used for model evaluation
# def compare_user_similarity(user_ingredients_list, products_list):
#     # make sure that it's a list
#     ingredients = ast.literal_eval(user_ingredients_list)
#     ingredients = [n.strip() for n in ingredients]
#     result_list = [] # store the smiles representations in a list
#     # turn the user's ingredients into smile representations
#     for i in ingredients:
#         result = cirpy.resolve(f'{i}', 'smiles')
#         if result is None:
#             continue
#         result_list.append(result)

#     # Convert SMILES to RDKit Molecule objects 
#     user_fingerprint_list = []
#     for r in result_list:
#         mol = Chem.MolFromSmiles(r)
#         try:
#             # Generate Morgan Fingerprints
#             user_fingerprint_list.append(AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048))
#         except:
#             continue
    
#     total_similarity = calculate_sum_similarity()

In [29]:
user_smiles = ["OCC(O)CO", "CCCCCCCCCCCCCCCCO"]
# turn into fingerprints
user_fingerprint_list = []
for r in user_smiles:
        mol = Chem.MolFromSmiles(r)
        try:
            # Generate Morgan Fingerprints
            user_fingerprint_list.append(AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048))
        except:
            continue
total_similarity = calculate_top_5_similar_products(user_fingerprint_list, merged_df) # this returns correct normalized similarities
total_similarity

{'Avène Tolérance Extrême Cream 50ml': 0.5789473684210527,
 'BeautyPro CBD Oil Infused Sheet Mask 22ml': 0.49689126662810873,
 'Avène Tolérance Extrême Emulsion 50ml': 0.4183723196881092,
 'La Roche-Posay Hydraphase Intense Masque 50ml': 0.3642790234763773,
 'Garnier Micellar Water Facial Cleanser and Makeup Remover for\xa0Sensitive Skin 700ml': 0.3543592116031279}